# Utils

> Module die een aantal praktische functies heeft om het programmeer leven makkelijker te maken.

In [1]:
#| default_exp utils

In [2]:
#| hide
from nbdev.showdoc import *

In [5]:
#| export
import pickle
import re
import yaml
import logging

from functools import wraps
from datetime import datetime
from fastcore.utils import Path

Set up logging

In [6]:
#| export

def setup_logging(output_dir: str, current_date: str = None) -> logging.Logger:
    if current_date is None:
        current_date = datetime.now().strftime('%Y-%m-%d')
    
    # Create log file path
    log_file = Path(output_dir) / f'logging_{current_date}.log'
    
    # Reset any existing handlers
    logger = logging.getLogger(__name__)
    logger.handlers = []
    
    # Create and configure file handler
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(logging.INFO)
    
    # Create and set formatter
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s',
                                datefmt='%Y-%m-%d %H:%M:%S')
    file_handler.setFormatter(formatter)
    
    # Configure logger
    logger.setLevel(logging.INFO)
    logger.addHandler(file_handler)
    
    return logger

Get project root

In [36]:
#| export
def get_project_root():
    current = Path.cwd()
    while current.name != current.parent:
        if (current / 'setup.py').exists() or (current / 'pyproject.toml').exists():
            return current
            raise FileNotFoundError('Could not find project root')
        current = current.parent
    return None

In [38]:
#| hide
get_project_root()

Path('/home/jelle/code/optimalisatie_route_peilbuizen')

Helper function to load settings into a dictionary.

In [40]:
#| export
def load_settings(settings_yaml: str = 'settings.yaml'): 
    project_root = get_project_root()

    with open(project_root / 'settings.yaml', 'r') as f:
        settings = yaml.safe_load(f)
    return settings

Functie om willekeurige text om te zetten naar text die probleemloos als filename gebruikt kan worden.

In [15]:
#| export

def make_filesystem_friendly(text: str) -> str:
    """Convert text to filesystem-friendly format
    Replaces any sequence of non-alphanumeric chars with single underscore"""
    # Replace non-alphanumeric characters with underscore
    text = re.sub(r'[^a-zA-Z0-9]+', '_', text)
    # Remove leading/trailing underscores
    text = text.strip('_')
    # Convert to lowercase
    return text.lower()

Functie die als een wrapper aan een willekeurige bestaande functie wordt toegevoegd. En de resultaten van die
functie wegschrijft naar een pickle file met de naam van de functie en de datum tijd waarop de functie
is uitgevoerd.

In [41]:
#| export

def funcresults_to_pickle(path_pickle):
    # A wrapper to add the ability to write the results of a function to a pickle file.
    # Simply add the decorator above the function and then add the parameter
    # save_to_pickle=True to the function call.
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Extract pickle parameter if present, default to False
            # pop the 'pickle' parameter so it doesn't get passed to the wrapped function unexpectedly.
            save_pickle = kwargs.pop('save_to_pickle', False)

            print(f"save_to_pickle: {save_pickle}")

            result = func(*args, **kwargs)
            
            # Check if path_pickle exists
            # Don't create the directory if it doesn't exist, to avoid unnecessary save of files in directories unknown to user
            if not Path(path_pickle).is_dir():
                raise ValueError(f"""'{path_pickle}' is not a valid directory. Give a valid directory path to save the pickle results.\n
                                 This can be changed in the parameters.py file.""")

            if save_pickle:
                print(f"Saving results to {path_pickle}")
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                pickle_filename = make_filesystem_friendly(f"{func.__name__}_{timestamp}")
                pickle_filename = pickle_filename + ".pickle"
                filepath = Path(path_pickle) / pickle_filename
                with open(filepath, 'wb') as f:
                    pickle.dump(result, f)
            
            return result
        return wrapper
    return decorator

Functie om URL shortcut bestanden te maken.

In [3]:
#| export

def save_route_url(url: str, filepath: str, filename: str):
    """Save URL as a Windows-compatible shortcut file"""
    path = Path(filepath)
    path.mkdir(parents=True, exist_ok=True)
    
    if not filename.endswith('.url'):
        filename = filename + '.url'
    
    content = f"""[InternetShortcut]
URL={url}"""

    with open(path / filename, 'w') as f:
        f.write(content)

In [1]:
#| export

save_route_url('https://www.google.com', './data', 'googletest.url')

NameError: name 'save_route_url' is not defined

In [6]:
#| hide
import nbdev; nbdev.nbdev_export()